In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

### Carregar dados e visualizar -  Regressao

In [2]:
# Carregar dados
df = pd.read_csv('./datasets/colesterol.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [4]:
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23
5,6,B,Não,Baixo,68,66.8,170,185.31
6,7,A,Sim,Baixo,60,117.3,181,289.33
7,8,O,Sim,Moderado,35,86.9,174,216.48
8,9,O,Não,Baixo,62,81.3,166,235.30
9,10,B,Sim,Alto,44,32.7,165,97.79


In [5]:
# Ajustar dataframe
df.drop('Id', axis=1, inplace=True)

#aplicar one hot nas variaveis categoricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [6]:
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,False,False,True,False,False,True,False,True,False
1,68,105.0,184,236.98,True,False,False,False,True,False,False,False,True
2,25,64.8,180,161.79,False,False,False,True,True,False,True,False,False
3,43,120.2,167,336.24,True,False,False,False,True,False,True,False,False
4,79,88.5,175,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,68.1,166,206.81,False,True,False,False,False,True,False,False,True
996,51,47.7,170,128.03,False,False,False,True,True,False,True,False,False
997,39,85.5,176,211.14,False,True,False,False,True,False,False,True,False
998,61,91.2,161,284.53,False,True,False,False,False,True,False,True,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        1000 non-null   int64  
 1   Peso                         1000 non-null   float64
 2   Altura                       1000 non-null   int64  
 3   Colesterol                   1000 non-null   float64
 4   Grupo Sanguíneo_A            1000 non-null   bool   
 5   Grupo Sanguíneo_AB           1000 non-null   bool   
 6   Grupo Sanguíneo_B            1000 non-null   bool   
 7   Grupo Sanguíneo_O            1000 non-null   bool   
 8   Fumante_Não                  1000 non-null   bool   
 9   Fumante_Sim                  1000 non-null   bool   
 10  Nível de Atividade_Alto      1000 non-null   bool   
 11  Nível de Atividade_Baixo     1000 non-null   bool   
 12  Nível de Atividade_Moderado  1000 non-null   bool   
dtypes: bool(9), float64

### Treinar modelo - Regressão com lasso

In [8]:
#Separar x e y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [9]:
#Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [11]:
#Treinar o modelo de regressao linear multipla com Lasso
#Quanto maior o apha maior a penalização e mais coeficientes tendem a ser reduzidos a zero
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [12]:
#mostrar importancia de features 
def importancia_features(modelo):
    importance = np.abs(modelo.coef_)
    print('Importancia das features')
    for i, feature in enumerate(modelo.feature_names_in_):
        print(f"{feature}: {importance[i]}")

In [13]:
#importando lasso
importancia_features(model_lasso)

Importancia das features
Idade: 0.017349176463551177
Peso: 2.4911793204672508
Altura: 2.20863191913672
Grupo Sanguíneo_A: 0.05063677331183119
Grupo Sanguíneo_AB: 1.032901896958969
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.6422787796520806
Fumante_Sim: 0.0
Nível de Atividade_Alto: 1.485309859678915
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.17448169861080404


In [14]:
#Função para avaliar performace

def performace_regressao(modelo, X_test, y_test):
    #faz a predição com modelo no conjunto de testes
    y_pred = modelo.predict(X_test)
    #Avaliar desempenho
    rmse = mean_squared_error(y_test, y_pred)
    mean_squared = np.sqrt(rmse)
    return mean_squared

In [15]:
# Performace regressao lasso
performace_regressao(model_lasso, X_test, y_test)

np.float64(8.96623471437868)

In [16]:
# Treinar com lassoCV
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X_train, y_train)


LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)

In [17]:
importancia_features(model_lasso_cv)

Importancia das features
Idade: 0.017349176463551177
Peso: 2.4911793204672508
Altura: 2.20863191913672
Grupo Sanguíneo_A: 0.05063677331183119
Grupo Sanguíneo_AB: 1.032901896958969
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.6422787796520806
Fumante_Sim: 0.0
Nível de Atividade_Alto: 1.485309859678915
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.17448169861080404


In [19]:
performace_regressao(model_lasso_cv, X_test, y_test)

np.float64(8.96623471437868)

### Treinar modelo - Regressão com Ridge

In [21]:
#Treinar o modelo de regressao linear multipla com Ridge
#Quanto maior o apha maior a penalização e mais coeficientes tendem a ser reduzidos a zero
model_Ridge = Ridge(alpha=0.1)
model_Ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [23]:
#importando Ridge
importancia_features(model_Ridge)

Importancia das features
Idade: 0.014856795381590114
Peso: 2.4863676267885966
Altura: 2.2052086366942296
Grupo Sanguíneo_A: 0.6068720485512523
Grupo Sanguíneo_AB: 1.2429953371312654
Grupo Sanguíneo_B: 0.8279752477160831
Grupo Sanguíneo_O: 0.1918519591557123
Fumante_Não: 1.0743689137577013
Fumante_Sim: 1.0743689137461385
Nível de Atividade_Alto: 1.4197372174686418
Nível de Atividade_Baixo: 0.42678204861783975
Nível de Atividade_Moderado: 0.992955168899216


In [ ]:
# Performace regressao Ridge
performace_regressao(model_Ridge, X_test, y_test)

np.float64(9.017609728696327)

In [26]:
# Treinar com RidgeCV
model_Ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1], cv=5)
model_Ridge_cv.fit(X_train, y_train)


RidgeCV(alphas=[0.1, 0.5, 1], cv=5)

In [27]:
importancia_features(model_Ridge_cv)

Importancia das features
Idade: 0.014895951901827532
Peso: 2.48649280110136
Altura: 2.2052934333900227
Grupo Sanguíneo_A: 0.6046814875079645
Grupo Sanguíneo_AB: 1.2362348914740726
Grupo Sanguíneo_B: 0.8213793355966116
Grupo Sanguíneo_O: 0.1898259316329705
Fumante_Não: 1.0708677672836826
Fumante_Sim: 1.0708677672837739
Nível de Atividade_Alto: 1.4132391996703664
Nível de Atividade_Baixo: 0.425092079911677
Nível de Atividade_Moderado: 0.9881471197628294


In [28]:
performace_regressao(model_Ridge_cv, X_test, y_test)

np.float64(9.017032321420583)